In [2]:
import os
import numpy
import sys
from astropy.coordinates import SkyCoord
import astropy.units as u
import mechanize
#import montage_wrapper as montage
from MontagePy.main import mSubimage, mViewer
from astropy.coordinates import ICRS, Galactic, FK4, FK5
import astropy.io.fits as fits

ModuleNotFoundError: No module named 'MontagePy'

In [3]:
#========================================================================
# Written by K. Nyland
# Last edited 11/14/19
#========================================================================
#exec(open('get_VLASS_cutouts.py').read())


# NOTES:
# 1)Download VLASS tile summary here: https://archive-new.nrao.edu/vlass/VLASS_dyn_summary.php
# 2) Fix inconsistent number of columns (the word "imaged" in the last column behaves as a separate column with no header)
#    by loading .php file into Excel and removing the column with the word "imaged."

#========================================================================
# Set input/output files and directories
VLASS_tile_catalog = 'VLASS_dyn_summary.txt'
source_catalog = '/Users/knyland/Desktop/VLASS/REALFAST/Topcat/VLASS_realfast_IPS_candidates_VLASS.fits'

outfile_summary = 'VLASS_matches_summary_v2.txt'

fits_dir = '/Users/knyland/Desktop/VLASS/REALFAST/VLASS/fits/VLASS1/quicklook'
if not os.path.exists(fits_dir):
	os.makedirs(fits_dir)

fits_dir_cutouts = './fits/VLASS1/quicklook/cutouts'
if not os.path.exists(fits_dir_cutouts):
	os.makedirs(fits_dir_cutouts)

rootname = 'VLASS2'
version = '1'
radius = 3.0 # cutout radius in arcmin

#========================================================================
# Load input files
tiles = numpy.genfromtxt(VLASS_tile_catalog,delimiter='',skip_header=2,dtype=str,usecols=0)
dec_min = numpy.genfromtxt(VLASS_tile_catalog,delimiter='',skip_header=2,dtype=float,usecols=1)
dec_max = numpy.genfromtxt(VLASS_tile_catalog,delimiter='',skip_header=2,dtype=float,usecols=2)
ra_min = numpy.genfromtxt(VLASS_tile_catalog,delimiter='',skip_header=2,dtype=float,usecols=3)
ra_max = numpy.genfromtxt(VLASS_tile_catalog,delimiter='',skip_header=2,dtype=float,usecols=4)
epoch = numpy.genfromtxt(VLASS_tile_catalog,delimiter='',skip_header=2,dtype=str,usecols=5)
date = numpy.genfromtxt(VLASS_tile_catalog,delimiter='',skip_header=2,dtype=str,usecols=6)
status = numpy.genfromtxt(VLASS_tile_catalog,delimiter='',skip_header=2,dtype=str,usecols=7)

hdu = fits.open(source_catalog)
data = hdu[1].data
sources = data['CandId']
ra_source = data['RA_1']
dec_source = data['DEC_1']
tiles = data['Tile']
subtiles = data['Subtile']

#========================================================================
coor_source = SkyCoord(ra_source*u.deg, dec_source*u.deg)
hour = coor_source.ra.hour

# Match source positions against tile coordinate limits
ra_deg = coor_source.ra.value
dec_deg = coor_source.dec.value

names = []; epochs = []
for i in range(0,len(coor_source)):
	ra_name = "%02d" % int(coor_source[i].ra.hms[0]) + "%02d" % int(coor_source[i].ra.hms[1])
	dec_name = "%02d" % int(coor_source[i].dec.signed_dms[1]) + "%02d" % int(coor_source[i].dec.signed_dms[1])
	sign = "%01d" % int(coor_source[i].dec.signed_dms[0])
	if sign == '-1':
		names.append('J' + ra_name + '-' + dec_name)
	elif sign == '1':
		names.append('J' + ra_name + '+' + dec_name)
	epochs.append(sources[i].split('.sb')[0])

'''
tile_matches = []; epoch_matches = []; date_matches = []
for i in range(0,len(names)):
	"%02d" % int(coor_source[i].ra.hms[0])
	hour_value = hour[i]
	dec_value = dec_deg[i]
	mask_index = numpy.where((hour_value > ra_min) & (hour_value < ra_max) & (dec_value > dec_min) & (dec_value < dec_max))[0][0]
	tile_matches.append(tiles[mask_index])
	epoch_matches.append(epoch[mask_index])
	date_matches.append(date[mask_index])
'''
#tile_matches_unique = list(set(tile_matches))

#========================================================================
# Select image with smallest separation
images = []; seps_all = []; image_links_all = []
for i in range(0,len(names)):
	print(i)
	print('Name: ', names[i])
	#seps = []
	#br = mechanize.Browser()
	#br.set_handle_robots(False)
	#if epoch_matches[i] == 'VLASS1.1':
	#	epoch_name = 'VLASS2.1'
	#elif epoch_matches[i] == 'VLASS1.2':
	#	epoch_name = 'VLASS2.2'
	epoch_name = epochs[i]
	#tile = tile_matches[i]
	tile = tiles[i]
	subtile = subtiles[i]
	imagename = epoch_name + '.ql.' + tile + '.' + subtile + '.10.2048.v1.I.iter1.image.pbcor.tt0.subim.fits'
	print(imagename)
	images.append(imagename)

#========================================================================
'''
# Get image name for download
image_files = []
for i in range(0,len(images)):
	br = mechanize.Browser()
	br.set_handle_robots(False)
	if epoch_matches[i] == 'VLASS1.1':
		epoch_name = 'VLASS2.1'
	elif epoch_matches[i] == 'VLASS1.2':
		epoch_name = 'VLASS2.2'
	#tile = tile_matches[i]
	tile = tiles[i]
	subtile = subtiles[i]
	image = images[i]
	#https://archive-new.nrao.edu/vlass/quicklook/VLASS2.2/T22t07/VLASS2.2.ql.T22t07.J064715+443000.10.2048.v1/
	#myurl = 'https://archive-new.nrao.edu/vlass/quicklook/' + epoch_name + '/' + tile + '/' + epoch_name + '.ql.' + tile + '.' + subtile + '.10.2048.v1 + '/' + image
	myurl = 'https://archive-new.nrao.edu/vlass/quicklook/' + epoch_name + '/' + tile + '/' + epoch_name + '.ql.' + tile + '.' + subtile + '.10.2048.v1/'
	image_files.append(myurl + image)
	br.open(myurl)
	for link in br.links():
		#print(link.text)
		if '.tt0.subim.fits' in link.text:
			link = str(link.text)
			file = myurl + link
			print(link)
			print(file)
			image_names.append(link)
			image_files.append(file)
'''


sources_data = []; names_data = []; ra_deg_data = []; dec_deg_data = []; epochs_data = []; date_data = []; time_data = []; tile_data = []; images_data = []; missing = []
date_beg = []; date_end = []
for i in range(0,len(sources)):
	fitsfile = fits_dir + '/' + images[i]
	print(fitsfile)
	if os.path.exists(fitsfile):
		sources_data.append(sources[i])
		names_data.append(names[i])
		ra_deg_data.append(str(ra_deg[i]))
		dec_deg_data.append(str(dec_deg[i]))
		hdu = fits.open(fitsfile)
		imhdr = hdu[0].header
		date_data.append(imhdr['DATE-OBS'].split('T')[0])
		time_data.append(imhdr['DATE-OBS'].split('T')[1])
		date_beg.append(imhdr['DATE-OBS'])
		date_end.append(imhdr['DATE-END'])
		images_data.append(images[i])
		epochs_data.append(epochs[i])
		tile_data.append(tiles[i])
		hdu.close()
	else:
		missing.append(sources[i])

outfile_summary4 = 'VLASS_matches_summary_all_v4.txt'
output_summary4 = list(zip(sources_data, names_data, ra_deg_data, dec_deg_data, epochs_data, date_beg, date_end, tile_data, images_data))
os.system('rm -rf ' + outfile_summary4)
f = open(outfile_summary4, "w")
thisrow = '#Source'.ljust(30) + 'Name'.rjust(14) + 'RA'.rjust(16) + 'DEC'.rjust(18) + 'Epoch'.rjust(12) + 'Date_beg'.rjust(30) + 'Date_end'.rjust(30) + 'Tile'.rjust(10) + 'File'.rjust(82) + ' \n'
print(thisrow)
f.writelines(thisrow)
for i in output_summary4:
    thisrow = i[0].ljust(30) + i[1].rjust(14) + i[2].rjust(16) + i[3].rjust(18) + i[4].rjust(12) + i[5].rjust(30) + i[6].rjust(30) + i[7].rjust(10) + i[8].rjust(82) +  ' \n'
    print(thisrow)
    f.writelines(thisrow)

f.close()

#========================================================================
# Download the fits files using wget
for i in range(0,len(image_files)):
#for i in range(303,len(image_files)):
	print(i)
	print(image_files[i])
	command = 'wget -P ' + fits_dir + ' ' + image_files[i]
	print(command)
	os.system(command)

#========================================================================
# Extract cutouts
hduext = 0
xwidth = 2*radius/60 # deg
ywidth = xwidth

missing = []
for i in range(0,len(sources)):
#for i in range(303,len(sources)):
	infile = os.path.join(fits_dir, images[i])
	ra = ra_deg[i]
	dec = dec_deg[i]
	outfile = fits_dir_cutouts + '/' + names[i] + '_' + rootname + '_' + 'cutout' + '_v' + version + '.fits'
	print(outfile)
	if not os.path.isfile(infile):
		error_message = 'ERROR: FITS FILE {} NOT FOUND - ABORTING SCRIPT'.format(infile)
		#sys.exit(error_message)
		missing.append(sources[i])
	else:
		#montage.commands.mSubimage(infile, outfile, ra, dec, xwidth, hdu=hduext, ysize=ywidth, status_file='montage.log', debug=False)
		mSubimage(infile, outfile, ra, dec, xwidth, hdu=hduext, ysize=ywidth)

PermissionError: [WinError 5] Access is denied: '/Users/knyland'